# Image Classification

In [4]:
from tensorflow.keras.utils import to_categorical
# Prep pixels for tfds datasets load dataset
def prep_pixels2(train, test, target_train, target_test):
    img_rows=28
    img_cols=28
    #if k.image_data_format() == 'channels_first':
    #X_train = train.reshape(train.shape[0], 1, img_rows, img_cols)
    #X_test = test.reshape(test.shape[0], 1, img_rows, img_cols)
    #input_shape = (1, img_rows, img_cols)
    #else:
    X_train = train.reshape(train.shape[0], img_rows, img_cols, 1)
    X_test = test.reshape(test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    target_train = to_categorical(target_train)
    target_test =  to_categorical(target_test)
    return train_norm, test_norm, ta

In [5]:
import cv2
import pandas
import sklearn
import tensorflow as tf
from sklearn.model_selection import KFold

In [6]:
#import tensorflow as tf
from tensorflow.keras.utils import to_categorical
# Prep pixels for tfds datasets load dataset
def prep_pixels(image, label):
    img_rows=28
    img_cols=28
    image = tf.cast(image, tf.float32)
    image = tf.divide(image, 255)
    train_norm = tf.image.resize(image, (32, 32))
    target = tf.one_hot(label, depth=10)
    return train_norm, target

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import SimpleRNN, LSTM, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.metrics import categorical_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.layers import BatchNormalization, Activation

# CNN model
def val_cnn_model(n_channels=1):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(32, 32, n_channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(320, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    opt = SGD(learning_rate=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(loss=categorical_crossentropy, optimizer=Adadelta(), metrics=['accuracy'])
    return model

In [8]:
# CNN optimized for MNIST
def val_cnn_mnist(n_channels=1):
    model = Sequential()
    model.add(Conv2D(6, (5, 5), activation='relu', kernel_initializer='he_uniform', input_shape=(32, 32, n_channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(84, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    opt = SGD(learning_rate=0.1, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    '''model=Sequential()

    #model.add(Lambda(standardize,input_shape=(28,28,1)))    
    model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,n_channels)))
    model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
    model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())    

    model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(512,activation="relu"))

    model.add(Dense(10,activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])'''
    return model

In [46]:
from tensorflow.keras import regularizers
#data augmentation
'''
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)'''

# CNN Optimized for CIFAR10
def val_cnn_cifar(n_channels=3):
    weight_decay = 1e-4
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(32, 32, n_channels)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    opt=RMSprop(lr=0.001,decay=1e-6)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [89]:
# RNN network for images
def val_rnn_model(x_train):
    '''i = Input(shape=x_train[0].shape)
    x = LSTM(128)(i)
    x = Dense(10, activation='softmax')(x)
    model=Model(i, x)'''
    
    model=Sequential()
    #model.add(Input(shape=x_train[0].shape))
    model.add(LSTM(128, input_shape=x_train[0].shape))
    model.add(Dense(10, activation='softmax'))
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #model.compile(loss=categorical_crossentropy, optimizer=Adadelta(), metrics=['accuracy'])
    return model

In [6]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model

# Pretrained MobileNet network for image recognition
def val_mn_model(n_channels=3):
    bottom_model = MobileNet(weights='imagenet', include_top=False, input_shape=(32,32, n_channels))
    for layer in bottom_model.layers:
        layer.trainable = False
    top_model = Flatten(name='flatten')(bottom_model.output)#top_model = Dense(1024, activation='relu')(bottom_model.output)
    top_model = Dense(10, activation='relu')(top_model)
    top_model = Dense(10, activation='softmax')(top_model)
    model = Model(inputs = bottom_model.inputs, outputs=top_model)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [58]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Pretrained VGG Network for image recognition
def val_vgg_model(n_channels=3):
    bottom_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32, n_channels))
    for layer in bottom_model.layers:
        layer.trainable = False
    top_model = Flatten(name='flatten')(bottom_model.output)
    top_model = Dense(10, activation='relu')(top_model)
    top_model = Dense(10, activation='softmax')(top_model)
    model = Model(inputs = bottom_model.inputs, outputs=top_model)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [55]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model

# Pretrained ResNet Network for image recognition
def val_resnet_model(n_channels=3):
    bottom_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32,32, n_channels))
    for layer in bottom_model.layers:
        layer.trainable = False
    top_model = Flatten(name='flatten')(bottom_model.output)
    top_model = Dense(10, activation='relu')(top_model)
    top_model = Dense(10, activation='softmax')(top_model)
    model = Model(inputs = bottom_model.inputs, outputs=top_model)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
def evaluate_image_model(train, test, n_channels=3, n_folds=5):
    scores, histories = list(), list()
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    model=val_cnn_mnist(n_channels) # 3 epochs
    #model=val_cnn_cifar(n_channels) #Needs 80pct accuracy Eurosat gets > 80 at epoch 2 and overfits afterwards
    #Needs 80pct accuracy CIFAR10 gets > 80 at epoch 5 for train and epoch 6 for validation. Each epoch is 12 min
    #model=val_vgg_model(n_channels) # Reached 80pct at epoch 6. 10 minutes per epoch
    model.fit(train, batch_size=128, epochs=10, steps_per_epoch=60000 // 64, validation_data=test, verbose=2)
    _, acc = model.evaluate(test, verbose=2)
    scores.append(acc)
    return scores

In [10]:
# Train on RNN network
def evaluate_image_model_rnn(x_train, y_train, x_test, y_test, n_folds=5):
    scores, histories = list(), list()
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    model=val_rnn_model(x_train)
    
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, verbose=2)
    _, acc = model.evaluate(x_test, y_test, verbose=2)
    scores.append(acc)
    return scores

In [11]:
#(X_train, Y_train) = train_ds
#(X_test, Y_test) = test_ds

In [13]:
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
# Load datasets
#train_ds, test_ds=tf.keras.datasets.mnist.load_data()
#train_ds, test_ds = tfds.load('eurosat', split=['train[:75%]','train[75%:]'], as_supervised=True)
train_ds, test_ds = tfds.load('mnist', split=['train','test'], as_supervised=True) # 3 Epochs
#train_ds, test_ds = tfds.load('cifar10', split=['train[:75%]','train[75%:]'], as_supervised=True)
train = train_ds.map(prep_pixels).cache().shuffle(100).batch(64).repeat()
test = test_ds.map(prep_pixels).cache().batch(64)
evaluate_image_model(train, test, n_channels=1)

Epoch 1/10
937/937 - 17s - loss: 0.2407 - accuracy: 0.9301 - val_loss: 0.1004 - val_accuracy: 0.9719
Epoch 2/10
937/937 - 12s - loss: 0.1135 - accuracy: 0.9704 - val_loss: 0.1080 - val_accuracy: 0.9742
Epoch 3/10
937/937 - 13s - loss: 0.1119 - accuracy: 0.9724 - val_loss: 0.0990 - val_accuracy: 0.9761
Epoch 4/10
937/937 - 12s - loss: 0.0902 - accuracy: 0.9775 - val_loss: 0.1166 - val_accuracy: 0.9717
Epoch 5/10
937/937 - 13s - loss: 0.0947 - accuracy: 0.9775 - val_loss: 0.1372 - val_accuracy: 0.9640
Epoch 6/10
937/937 - 12s - loss: 0.1013 - accuracy: 0.9774 - val_loss: 0.1409 - val_accuracy: 0.9685
Epoch 7/10
937/937 - 13s - loss: 0.1277 - accuracy: 0.9737 - val_loss: 0.1764 - val_accuracy: 0.9721
Epoch 8/10
937/937 - 12s - loss: 0.4645 - accuracy: 0.8570 - val_loss: 2.1209 - val_accuracy: 0.1808
Epoch 9/10
937/937 - 12s - loss: 2.3037 - accuracy: 0.1067 - val_loss: 2.3036 - val_accuracy: 0.1009
Epoch 10/10
937/937 - 13s - loss: 2.3050 - accuracy: 0.1054 - val_loss: 2.3032 - val_accura

[0.10090000182390213]

In [98]:
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.mnist.load_data()
x_train = x_train/255.0
x_test=x_test/255.0
#train = train_ds.map(prep_pixels).cache().shuffle(100).batch(64).repeat()
#test = test_ds.map(prep_pixels).cache().batch(64)
evaluate_image_model_rnn(x_train, y_train, x_test, y_test)

Epoch 1/10
1875/1875 - 59s - loss: 0.5603 - accuracy: 0.8167 - val_loss: 0.1987 - val_accuracy: 0.9404
Epoch 2/10
1875/1875 - 68s - loss: 0.1446 - accuracy: 0.9565 - val_loss: 0.1134 - val_accuracy: 0.9670
Epoch 3/10
1875/1875 - 63s - loss: 0.0965 - accuracy: 0.9707 - val_loss: 0.0931 - val_accuracy: 0.9722
Epoch 4/10
1875/1875 - 82s - loss: 0.0745 - accuracy: 0.9782 - val_loss: 0.0747 - val_accuracy: 0.9785
Epoch 5/10
1875/1875 - 92s - loss: 0.0607 - accuracy: 0.9819 - val_loss: 0.0598 - val_accuracy: 0.9820
Epoch 6/10
1875/1875 - 96s - loss: 0.0504 - accuracy: 0.9845 - val_loss: 0.0569 - val_accuracy: 0.9828
Epoch 7/10
1875/1875 - 89s - loss: 0.0439 - accuracy: 0.9872 - val_loss: 0.0537 - val_accuracy: 0.9835
Epoch 8/10
1875/1875 - 98s - loss: 0.0383 - accuracy: 0.9880 - val_loss: 0.0489 - val_accuracy: 0.9847
Epoch 9/10
1875/1875 - 74s - loss: 0.0322 - accuracy: 0.9904 - val_loss: 0.0478 - val_accuracy: 0.9847
Epoch 10/10
1875/1875 - 70s - loss: 0.0293 - accuracy: 0.9908 - val_loss:

[0.9830999970436096]

In [12]:
import cv2
import pandas
import sklearn
import tensorflow as tf
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from scipy import ndimage
from sklearn.model_selection import KFold
import tensorflow as tf
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def val_load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=10) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=10)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

# NLP

In [178]:
import sklearn
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.regularizers import l1

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

# LSTM model for NLP
def lstm_sequence_model(maxlen,
                   max_features,
                   embed_size,
                   embedding_matrix,
                   metrics):
    
    tf.keras.backend.clear_session()
    
    if embedding_matrix is not None:
        embeddings = [embedding_matrix]
        output_dim = embedding_matrix.shape[1]
    else:
        embeddings = None

    model = Sequential()
    model.add(Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True))
    model.add(Reshape((maxlen, embed_size)))
    model.add(Bidirectional(LSTM(27, return_sequences = True)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(20, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    '''model = tf.keras.models.Sequential([
        #tf.keras.layers.Input(shape=maxlen),
        tf.keras.layers.Embedding(max_features, 
                                  embed_size, 
                                  weights=[embedding_matrix], 
                                  trainable=True),
        #tf.keras.layers.Reshape((max_len, embed_size))
        #tf.keras.layers.Bidirectional(
        #    tf.keras.layers.LSTM(27, activation='relu', return_sequences = True)),
        #tf.keras.layers.GlobalMaxPooling1D(),
        #tf.keras.layers.Dense(16, activation='relu'),
        #tf.keras.layers.Dense(1, activation='sigmoid')
    ])'''
    
    optimizer = tf.keras.optimizers.Adam(lr=0.1, epsilon=0.01)
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=metrics)
    
    return model



In [181]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import BinaryAccuracy

# Loading Glove Model
def load_glove_model(File):
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

# NLP classification code
def val_nlp_classification(X_train, X_test, Y_train, Y_test, n_folds):
    #set maxlen based on right edge of question length histogram 
    maxlen = 250

    #arbitrary choice of top 25000 words
    max_features=25000

    tokenizer = Tokenizer(num_words=max_features, oov_token="<oov>", filters='"!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n"', split=" ")

    tokenizer.fit_on_texts(np.concatenate([X_train, X_test]))
    train_df = tokenizer.texts_to_sequences(X_train)
    train_df = pad_sequences(train_df, maxlen=maxlen, padding="post", truncating='post')
    test_df = tokenizer.texts_to_sequences(X_test)
    test_df = pad_sequences(test_df, maxlen=maxlen, padding="post", truncating='post')


    EMBEDDING_FILE = 'C:/Users/vtoci/Documents/glove.6B.50d.txt'

    embeddings_index = dict()

    #open the embedding file
    with open(EMBEDDING_FILE, 'r', encoding="utf-8") as f:
        for line in f:
            values=line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float')
            embeddings_index[word] = coefs
        f.close()
    

    #glove_embedding_index = load_glove_model('glove.txt')

    # get the mean and standard deviation of the embeddings weights
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    #add the missing words to the embeddings and generate the random values
    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    METRICS = [
        tf.keras.metrics.AUC(name='roc-auc'),
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name="recall")
              ]
    val_metrics=BinaryAccuracy(name='accuracy')
    

    print(f"Maximum sequence length: {maxlen}")
    print(f"Number of words in the embedding: {max_features}")
    print(f"Number of words in the vocabulary: {len(tokenizer.word_index)}")
    print(f"Number of features per embedding: {embed_size}")
    #embedding_matrix = tf.convert_to_tensor(embedding_matrix)
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    model = lstm_sequence_model(maxlen, max_features, embed_size, embedding_matrix, val_metrics)
    model.fit(train_df, Y_train, epochs=10, batch_size=64, validation_data=(test_df, Y_test))
    scores = list()
    #, acc = model.evaluate(test_df, verbose=2)
    #scores.append(acc)
    #ss_predictions = ss.copy()
    #ss_predictions['prediction'] = preds
    return scores

In [182]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.neighbors import NearestNeighbors

# Generate word cloud to embed the model

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embedding(file):
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

def make_embedding_matrix(embedding, tokenizer, len_voc):
    all_embs = np.stack(embedding.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embedding.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

def make_tokenizer(texts, len_voc):
    t = Tokenizer(num_words=len_voc)
    t.fit_on_texts(texts)
    return t

def modify_sentence(sentence, synonyms, p=0.5):
    for i in range(len(sentence)):
        if np.random.random() > p:
            try:
                syns = synonyms[sentence[i]]
                sentence[i] = np.random.choice(syns)
            except KeyError:
                pass
    return sentence

def val_nlp_we_da(X, Y):
    synonyms_number = 5
    word_number = 20000
    np.random.seed(100)
    len_voc = 100000
    glove = load_embedding('C:/Users/vtoci/Documents/glove.6B.50d.txt')
    tokenizer = make_tokenizer(X, len_voc)
    train_df = tokenizer.texts_to_sequences(X)
    X = pad_sequences(train_df, maxlen=70, padding="post", truncating='post')
    #X = pad_sequences(X, 70)
    index_word = {0: ''}
    for word in tokenizer.word_index.keys():
        index_word[tokenizer.word_index[word]] = word
    embed_mat = make_embedding_matrix(glove, tokenizer, len_voc)
    nn = NearestNeighbors(n_neighbors=synonyms_number+1).fit(embed_mat) 
    neighbours_mat = nn.kneighbors(embed_mat[1:word_number])[1]
    synonyms = {x[0]: x[1:] for x in neighbours_mat}
    for x in np.random.randint(1, word_number, 10):
        print(f"{index_word[x]} : {[index_word[synonyms[x][i]] for i in range(synonyms_number-1)]}")
    X_pos = X[Y==1]
    indexes = np.random.randint(0, X_pos.shape[0], 10)
    for x in X_pos[indexes]:
        sample =  np.trim_zeros(x)
        sentence = ' '.join([index_word[x] for x in sample])

        modified = modify_sentence(sample, synonyms)
        sentence_m = ' '.join([index_word[x] for x in modified])

        print(' ')
    X_gen = np.array([modify_sentence(x, synonyms) for x in X_pos[indexes]])
    y_gen = np.ones(len(Y))
    return

In [183]:
import tensorflow_datasets as tfds
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

def evaluate_model(train, test, n_folds=5):
    scores, histories = list(), list()
    X_train, Y_train = tfds.as_numpy(train)
    X_test, Y_test = tfds.as_numpy(test)
    #Y_train = Y_train.astype('str').tolist()
    #Y_test = Y_test.astype('str').tolist()
    X_train=pd.DataFrame(X_train, columns=['Text'])
    X_train = X_train['Text'].str.decode("utf-8")
    X_test=pd.DataFrame(X_test, columns=['Text'])
    X_test = X_test['Text'].str.decode("utf-8")
    
    
    label_encoder = LabelEncoder()
    Y_train = np.array(label_encoder.fit_transform(Y_train))
    Y_test = np.array(label_encoder.fit_transform(Y_test))
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    #X_train, Y_train = val_nlp_we_da(X_train, Y_train)
    scores=val_nlp_classification(X_train, X_test, Y_train, Y_test, n_folds)
    return scores

def evaluate_data_augment_model(train, test, n_folds=5):
    scores, histories = list(), list()
    X_train, Y_train = tfds.as_numpy(train)
    X_test, Y_test = tfds.as_numpy(test)
    #Y_train = Y_train.astype('str').tolist()
    #Y_test = Y_test.astype('str').tolist()
    X_train=pd.DataFrame(X_train, columns=['Text'])
    X_train = X_train['Text'].str.decode("utf-8")
    X_test=pd.DataFrame(X_test, columns=['Text'])
    X_test = X_test['Text'].str.decode("utf-8")
    
    
    label_encoder = LabelEncoder()
    Y_train = np.array(label_encoder.fit_transform(Y_train))
    Y_test = np.array(label_encoder.fit_transform(Y_test))
    print(set(Y_test))
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    val_nlp_we_da(X_train, Y_train)
    return

train_ds, test_ds = tfds.load('imdb_reviews', split=['train', 'test'], batch_size=-1, as_supervised=True)
scores = evaluate_model(train_ds, test_ds)
#evaluate_data_augment_model(train_ds, test_ds)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


Maximum sequence length: 250
Number of words in the embedding: 25000
Number of words in the vocabulary: 124253
Number of features per embedding: 50
Epoch 1/10
391/391 [==============================] - 75s 191ms/step - loss: 0.5377 - accuracy: 0.6993 - val_loss: 0.3873 - val_accuracy: 0.8251
Epoch 2/10
391/391 [==============================] - 75s 191ms/step - loss: 0.3707 - accuracy: 0.8373 - val_loss: 0.3620 - val_accuracy: 0.8314
Epoch 3/10
391/391 [==============================] - 74s 189ms/step - loss: 0.3103 - accuracy: 0.8695 - val_loss: 0.3338 - val_accuracy: 0.8526
Epoch 4/10
391/391 [==============================] - 73s 188ms/step - loss: 0.2771 - accuracy: 0.8852 - val_loss: 0.3345 - val_accuracy: 0.8564
Epoch 5/10
391/391 [==============================] - 74s 188ms/step - loss: 0.2393 - accuracy: 0.9052 - val_loss: 0.3513 - val_accuracy: 0.8527
Epoch 6/10
391/391 [==============================] - 74s 190ms/step - loss: 0.2034 - accuracy: 0.9212 - val_loss: 0.3528 - val

## Text2Score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from tensorflow.keras.preprocessing import sequence, text
import tensorflow.keras.layers as layer 
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split



max_len = 30
tk = text.Tokenizer(num_words=50000)
tk.fit_on_texts(df_all['text'].str.lower().tolist())
X = tk.texts_to_sequences(df_all['text'].str.lower().values)
X = sequence.pad_sequences(X, maxlen=max_len)

df_train = df_all[df_all['deal_probability'].notnull()]
X_train, X_val, y_train, y_val  = train_test_split(X[:len(df_train)], df_train['deal_probability'].values, test_size=0.01)
'''

In [ ]:
# Model for sentiment analysis
def vtoc_sentiment_model(vocab_size, embed_size, max_len):
    inputs = tf.keras.Input(shape=(None,), dtype="int32")
    x = tf.keras.layers.Embedding(input_dim = vocab_size, 
                              output_dim = embed_size,
                              input_length=max_len)(inputs)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
    # Add a classifier
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs, outputs)

    # Compile and Run 
    model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])
    return model

def vectorize_sequences(train_data):
    training_sentences = []
    training_labels = []
    
    for sentence in train_data[0]:
        training_sentences.append(str(sentence.numpy()))
    for label in train_data[1]:
        training_labels.append(str(label.numpy()))
    return training_sentences, np.asarray(training_labels)

def evaluate_sentiment_model(train_ds, test_ds):
    X_train, Y_train = vectorize_sequences(train_ds)
    X_test, Y_test = vectorize_sequences(test_ds)
    #arbitrary choice of top 25000 words
    maxlen = 250
    max_features=25000
    vocab_size = 2000 # The maximum number of words to keep, based on word frequency. 
    embed_size = 30   # Dimension of the dense embedding.
    max_len = 100  
    tokenizer = Tokenizer(num_words=max_features, oov_token="<oov>", filters='"!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n"', split=" ")
    tokenizer.fit_on_texts(np.concatenate([X_train, X_test]))
    train_df = tokenizer.texts_to_sequences(X_train)
    train_padded = pad_sequences(train_df, maxlen=maxlen, padding="post", truncating='post')
    test_df = tokenizer.texts_to_sequences(X_test)
    test_padded = pad_sequences(test_df, maxlen=maxlen, padding="post", truncating='post')
    model=vtoc_sentiment_model(vocab_size, embed_size, max_len)
    model.summary()
    model.fit(train_padded, Y_train, validation_data=(test_padded, Y_test), epochs=5, verbose=3)
    return

# Other

In [19]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_datasets as tfds
# Basic ANN
class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self,epoch, logs=None):
        if logs['accuracy'] >0.90:
            print("Accuracy greater than 90%. Stopping Training.")
            self.model.stop_training=True
            
def val_dnn_model(epochs, X_train, Y_train, X_val, Y_val, callbacks=None):
    model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1, activation='sigmoid')])

    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, callbacks=callbacks)

    return history, model


In [20]:
from sklearn.model_selection import train_test_split
train = tfds.load('german_credit_numeric', split=['train'], batch_size=-1, as_supervised=True)
X=tfds.as_numpy(train[0][0])
Y=tfds.as_numpy(train[0][1])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3)
hist_regularized, model_regularized = val_dnn_model(100, X_train, Y_train, X_test, Y_test, callbacks=[EarlyStoppingCallback()])
print(f"Training Set:   {model_regularized.evaluate(X_train, Y_train)}")
print(f"Validation Set: {model_regularized.evaluate(X_test, Y_test)}")

Epoch 1/100
22/22 [==============================] - 0s 6ms/step - loss: 1.4741 - accuracy: 0.5600 - val_loss: 0.6924 - val_accuracy: 0.7000
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.9039 - accuracy: 0.6529 - val_loss: 0.5940 - val_accuracy: 0.7467
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.9534 - accuracy: 0.6171 - val_loss: 0.5701 - val_accuracy: 0.7367
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 0.7285 - accuracy: 0.6943 - val_loss: 0.5851 - val_accuracy: 0.7133
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.7493 - accuracy: 0.6571 - val_loss: 0.5599 - val_accuracy: 0.7333
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.7379 - accuracy: 0.6814 - val_loss: 0.5614 - val_accuracy: 0.7133
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 0.6848 - accuracy: 0.6857 - val_loss: 0.5483 - val_accuracy: 0.7433
Epoch 8/100
2

In [21]:
# Serialization
import tensorflow as tf
import numpy as np

PATH = '/kaggle/working/data.tfrecord'

with tf.io.TFRecordWriter(path=PATH) as f:
    f.write(b'123') # write one record
    f.write(b'xyz314') # write another record

with open(PATH, 'rb') as f:
    print(f.read())
    
x = tf.constant([[1, 2], [3, 4]], dtype=tf.uint8)
print('x:', x, '\n')

x_bytes = tf.io.serialize_tensor(x)
print('x_bytes:', x_bytes, '\n')

print('x:', tf.io.parse_tensor(x_bytes, out_type=tf.uint8))

"# Serialization\nimport tensorflow as tf\nimport numpy as np\n\nPATH = '/kaggle/working/data.tfrecord'\n\nwith tf.io.TFRecordWriter(path=PATH) as f:\n    f.write(b'123') # write one record\n    f.write(b'xyz314') # write another record\n\nwith open(PATH, 'rb') as f:\n    print(f.read())\n    \nx = tf.constant([[1, 2], [3, 4]], dtype=tf.uint8)\nprint('x:', x, '\n')\n\nx_bytes = tf.io.serialize_tensor(x)\nprint('x_bytes:', x_bytes, '\n')\n\nprint('x:', tf.io.parse_tensor(x_bytes, out_type=tf.uint8))"

# Timeseries

In [189]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

# Timeseries LSTM Autoencoder
def vtoc_lstm_autoencoder(n_steps, n_horizon, n_features, lr):
    serie_size=n_steps
    encoder_decoder = Sequential()
    encoder_decoder.add(LSTM(n_steps, activation='relu', input_shape=(n_steps, n_features), return_sequences=True))
    encoder_decoder.add(LSTM(6, activation='relu', return_sequences=True))
    encoder_decoder.add(LSTM(1, activation='relu'))
    encoder_decoder.add(RepeatVector(serie_size))
    encoder_decoder.add(LSTM(serie_size, activation='relu', return_sequences=True))
    encoder_decoder.add(LSTM(6, activation='relu', return_sequences=True))
    encoder_decoder.add(TimeDistributed(Dense(1)))
    encoder_decoder.summary()

    adam = Adam(lr)
    encoder_decoder.compile(loss='mse', optimizer=adam)
    return encoder_decoder

In [190]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *

#Timeseries LSTM Model
def vtoc_lstm_model(n_steps, n_horizon, n_features, lr):
    
    tf.keras.backend.clear_session()
    
    model = Sequential()
    model.add(LSTM(72, activation='relu', input_shape=(n_steps, n_features), return_sequences=True)),
    model.add(LSTM(48, activation='relu', return_sequences=True)),
    model.add(Flatten()),
    model.add(Dropout(0.3)),
    model.add(Dense(128, activation='relu')),
    model.add(Dropout(0.3)),
    model.add(Dense(n_horizon))
    
    loss = Huber()
    optimizer = Adam(lr=lr)
    
    model.compile(loss=loss, optimizer='adam', metrics=['mae'])
    
    return model

#Timeseries CNN Model
def vtoc_cnn_model(n_steps, n_horizon, n_features, lr=3e-4):
    
    tf.keras.backend.clear_session()
    
    model = Sequential()
    #tf.keras.layers.Input(shape=(n_steps, n_features)),
    #tf.keras.layers.Flatten(input_shape=(n_steps, n_features)),
    model.add(Conv1D(64, kernel_size=6, activation='relu', input_shape=(n_steps,n_features))),
    model.add(MaxPooling1D(2)),
    model.add(Conv1D(64, kernel_size=3, activation='relu')),
    model.add(MaxPooling1D(2)),
    model.add(Flatten()),
    #tf.keras.layers.Dropout(0.3),
    model.add(Dense(128)),
    model.add(Dropout(0.3)),
    model.add(Dense(n_horizon))
    
    loss= Huber()
    optimizer =Adam(lr=lr)
    
    model.compile(loss=loss, optimizer='adam', metrics=['mae'])
    
    return model

#Timeseries DNN Model
def vtoc_dnn_model(n_steps, n_horizon, n_features, lr):
    tf.keras.backend.clear_session()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(n_steps, n_features)), #Use with evaluate_timeseries_model
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(n_horizon)
    ], name='dnn')
    
    loss=tf.keras.losses.Huber()
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    model.compile(loss=loss, optimizer='adam', metrics=['mae'])
    
    return model

#Timeseries LSTM-CNN Model
def lstm_cnn_model(n_steps, n_horizon, n_features, lr):
    
    tf.keras.backend.clear_session()
    
    model = Sequential()
    model.add(Input(shape=(n_steps,n_features))),
    model.add(Conv1D(64, kernel_size=6, activation='relu', input_shape=(n_steps,n_features))),
    model.add(MaxPooling1D(2)),
    model.add(Conv1D(64, kernel_size=3, activation='relu')),
    model.add(MaxPooling1D(2)),
    model.add(LSTM(72, activation='relu', return_sequences=True)),
    model.add(LSTM(48, activation='relu', return_sequences=False)),
    model.add(Flatten()),
    model.add(Dropout(0.3)),
    model.add(Dense(128)),
    model.add(Dropout(0.3)),
    model.add(Dense(n_horizon))
    
    loss = Huber()
    optimizer = Adam(lr=lr)
    
    model.compile(loss=loss, optimizer='adam', metrics=['mae'])
    
    return model

#Timeseries LSTM-CNN Skip Model
def lstm_cnn_skip_model(n_steps, n_horizon, n_features, lr):
    
    tf.keras.backend.clear_session()
    
   
    inputs = tf.keras.layers.Input(shape=(n_steps,n_features), name='main')
    
    conv1 = tf.keras.layers.Conv1D(64, kernel_size=6, activation='relu')(inputs)
    max_pool_1 = tf.keras.layers.MaxPooling1D(2)(conv1)
    conv2 = tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu')(max_pool_1)
    max_pool_2 = tf.keras.layers.MaxPooling1D(2)(conv2)
    lstm_1 = tf.keras.layers.LSTM(72, activation='relu', return_sequences=True)(max_pool_2)
    lstm_2 = tf.keras.layers.LSTM(48, activation='relu', return_sequences=False)(lstm_1)
    flatten = tf.keras.layers.Flatten()(lstm_2)
    
    skip_flatten = tf.keras.layers.Flatten()(inputs)

    concat = tf.keras.layers.Concatenate(axis=-1)([flatten, skip_flatten])
    drop_1 = tf.keras.layers.Dropout(0.3)(concat)
    dense_1 = tf.keras.layers.Dense(128, activation='relu')(drop_1)
    drop_2 = tf.keras.layers.Dropout(0.3)(dense_1)
    output = tf.keras.layers.Dense(n_horizon)(drop_2)
    
    model = tf.keras.Model(inputs=inputs, outputs=output, name='lstm_skip')
    
    loss = tf.keras.losses.Huber()
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    model.compile(loss=loss, optimizer='adam', metrics=['mae'])
    
    return model

In [191]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import datetime

# Prepare timeseries data
def multi_baseline_eror(data, pred_cols):
    df = data[pred_cols]
    #fill nans with linear interpolation because this is how we will fill when using the data in the models.
    df_filled = df.interpolate("linear")
    mm = MinMaxScaler()
    df_scaled = mm.fit_transform(df_filled)
    df_prep = pd.DataFrame(df_scaled, columns=pred_cols)
    y_true = df_prep[pred_cols[0]]
    y_pred_forecast = df_prep[pred_cols[1]]

    ### persistence 1 day
    #shift series by 24 hours
    # realign y_true to have the same length and time samples
    y_preds_persistance_1_day = y_true.shift(24).dropna()
    persistence_1_day_mae = tf.keras.losses.MAE(y_true[y_preds_persistance_1_day.index], y_preds_persistance_1_day).numpy()
    persistence_1_day_mape = tf.keras.losses.MAPE(np.maximum(y_true[y_preds_persistance_1_day.index], 1e-5), np.maximum(y_preds_persistance_1_day, 1e-5)).numpy()


    ### persistence 3 day average
    #shift by 1, 2, 3 days. Realign to have same lengths. Average days and calcualte MAE.

    shift_dfs = list()
    for i in range(1, 4):
        shift_dfs.append(pd.Series(y_true.shift(24 * i), name=f"d{i}"))

    y_persistance_3d = pd.concat(shift_dfs, axis=1).dropna()
    y_persistance_3d["avg"] = (y_persistance_3d["d1"] + y_persistance_3d["d2"] + y_persistance_3d["d3"])/3
    d3_idx = y_persistance_3d.index
    persistence_3day_avg_mae = tf.keras.losses.MAE(y_true[d3_idx], y_persistance_3d['avg']).numpy()
    persistence_3day_avg_mape = tf.keras.losses.MAPE(np.maximum(y_true[d3_idx], 1e-5), np.maximum(y_persistance_3d['avg'], 1e-5)).numpy()


    ref_error = pd.DataFrame({
        "Method": ["TSO Forecast", "Persistence 1 Day", "Persitence 3 Day Avg"],
        "MAE": [tf.keras.losses.MAE(y_true, y_pred_forecast).numpy(),
                persistence_1_day_mae,
                persistence_3day_avg_mae],
        "MAPE":[tf.keras.losses.MAPE(np.maximum(y_true, 1e-5), np.maximum(y_pred_forecast, 1e-5)).numpy(),
                persistence_1_day_mape,
                persistence_3day_avg_mape]}, 
        index=[i for i in range(3)])
    return ref_error

def split_sequences(sequences, n_steps, extra_lag=False, long_lag_step=7, max_step=30, idx=0, multivar=False):
    #if not adding extra lag features adjust max_step and n_steps to aling
    if not extra_lag:
        max_step=n_steps
        n_steps+=1
        
    
    X, y = list(), list()
    for i in range(len(sequences)):
        
        # find the end of this pattern
        #end_ix = i + n_steps
        end_ix = i + max_step
        
        #create a list with the indexes we want to include in each sample
        slices = [x for x in range(end_ix-1,end_ix-n_steps, -1)] + [y for y in range(end_ix-n_steps, i, -long_lag_step)]
        
        #reverse the slice indexes
        slices = list(reversed(slices))
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break


        # gather input and output parts of the pattern
        seq_x = sequences[slices, :]
        seq_y = sequences[end_ix, :]

        X.append(seq_x)
        y.append(seq_y)
        
    X = np.array(X)
    y = np.array(y)
    
    if multivar:
        #unstack the 3rd dimension and select the first element(energy load)
        y = y[:,idx]
    
    return X, y
def split_data(series, train_fraq, test_len=8760):
    #slice the last year of data for testing 1 year has 8760 hours
    test_slice = len(series)-test_len

    test_data = series[test_slice:]
    train_val_data = series[:test_slice]

    #make train and validation from the remaining
    train_size = int(len(train_val_data) * train_fraq)
    
    train_data = train_val_data[:train_size]
    val_data = train_val_data[train_size:]
    
    return train_data, val_data, test_data

def create_window_data(dataset, look_back=1, n_features=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :n_features]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :n_features])
    return np.array(dataX), np.array(dataY)

def window_dataset(data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=False, expand_dims=False):
    #create a window with n steps back plus the size of the prediction length
    window = n_steps + n_horizon
    
    #expand dimensions to 3D to fit with LSTM inputs
    #creat the inital tensor dataset
    if expand_dims:
        ds = tf.expand_dims(data, axis=-1)
        ds = tf.data.Dataset.from_tensor_slices(ds)
    else:
        ds = tf.data.Dataset.from_tensor_slices(data)
    
    #create the window function shifting the data by the prediction length
    ds = ds.window(window, shift=n_horizon, drop_remainder=True)
    
    #flatten the dataset and batch into the window size
    ds = ds.flat_map(lambda x : x.batch(window))
    ds = ds.shuffle(shuffle_buffer)    
    
    #create the supervised learning problem x and y and batch
    if multi_var:
        ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:, :1]))
    else:
        ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:]))
    
    ds = ds.batch(batch_size).prefetch(1)
    
    return ds


def create_multi_dataset(data, pred_cols, multivar):
    date_time = pd.to_datetime(data.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    for column in pred_cols:
        # Getting rid of outliers
        data.loc[data[column] == -9999.0, column] = 0.0
    ref_error = multi_baseline_eror(timeseries_df, pred_cols)
    data=MinMaxScaler().fit_transform(data)
    tf.random.set_seed(42)
    train_fraq=0.65
    lr = 3e-4
    n_steps = 14#24*30
    n_horizon = 14
    batch_size = 64#256
    shuffle_buffer = 100
    if multivar:
        n_features=len(data[0])
    else:
        n_features=1
    train_data, val_data, test_data = split_data(data, train_fraq=train_fraq, test_len=8760)
    
    (X_train, Y_train)=create_window_data(train_data, look_back=n_horizon, n_features=n_features)
    (X_test, Y_test)=create_window_data(test_data, look_back=n_horizon, n_features=n_features)
    (X_val, Y_val)=create_window_data(val_data, look_back=n_horizon, n_features=n_features)
    #train_ds = window_dataset(train_data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multivar)
    #test_ds = window_dataset(test_data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multivar)
    #val_ds = window_dataset(val_data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multivar)
    #split_sequences(train_ds, n_steps, extra_lag=False, long_lag_step=7, max_step=30, idx=0, multivar=False)
    #print(f"Train Data Shape: {train_ds.shape}")
    #print(f"Val Data Shape: {val_ds.shape}")
    #kfold = KFold(n_folds=5, shuffle=True, random_state=1)
    model=vtoc_dnn_model(n_steps, n_horizon, n_features, lr)
    model.summary()
    model_hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=5, verbose=3)
    #model.fit(X_train, X_train, epochs=10, verbose=3, shuffle=True)
    model.evaluate(X_train, Y_train)
    validation_encoded = model.predict(X_val)
    #_, acc = model.evaluate(test, verbose=2)
    #scores.append(acc)
    return model_hist.history

def evaluate_multi_timeseries_model(timeseries_df, n_folds=5, multivar=False):
    df_processed = create_multi_dataset(timeseries_df, pred_cols = ['wv (m/s)', 'max. wv (m/s)'], multivar=multivar)
    print(df_processed)
    return

In [192]:
import os
import tensorflow as tf
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path) #We load the dataset in a csv_file



In [ ]:
import pandas as pd
timeseries_df = pd.read_csv(csv_path)
evaluate_multi_timeseries_model(timeseries_df, n_folds=5, multivar=True)

Model: "dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 196)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               25216     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                1806      
Total params: 43,534
Trainable params: 43,534
Non-trainable params: 0
___________________________________________________________

In [27]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import datetime

def create_dataset(X, y, delay=24, lookback=48):
    window_length = lookback + delay
    Xs, ys = [], []
    for i in range(lookback, len(X)-delay):
        v = X.iloc[i-lookback:i].to_numpy() # every one hour, we take the past 48 hours of features
        Xs.append(v)
        w = y.iloc[i+delay] # Every timestep, we take the temperature the next delay (here one day)
        ys.append(w)
    return(np.array(Xs), np.array(ys))

def preprocessing(data):
    
    # Getting rid of outliers
    data.loc[data['wv (m/s)'] == -9999.0, 'wv (m/s)'] = 0.0
    data.loc[data['max. wv (m/s)'] == -9999.0, 'max. wv (m/s)'] = 0.0
    
    # Taking values every hours
    data = data[5::6]# df[start,stop,step]
    
    wv = data.pop('wv (m/s)')
    max_wv = data.pop('max. wv (m/s)')

    # Convert to radians.
    wd_rad = data.pop('wd (deg)')*np.pi / 180

    # Calculate the wind x and y components.
    data['Wx'] = wv*np.cos(wd_rad)
    data['Wy'] = wv*np.sin(wd_rad)

    # Calculate the max wind x and y components.
    data['max Wx'] = max_wv*np.cos(wd_rad)
    data['max Wy'] = max_wv*np.sin(wd_rad)
    
    date_time = pd.to_datetime(data.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    
    day = 24*60*60 # Time is second within a single day
    year = 365.2425*day # Time in second withon a year

    data['Day sin'] = np.sin(timestamp_s * (2*np.pi / day))
    data['Day cos'] = np.cos(timestamp_s * (2*np.pi / day))
    data['Year sin'] = np.sin(timestamp_s * (2*np.pi / year))
    data['Year cos'] = np.cos(timestamp_s * (2*np.pi / year))
    
    return(data)

def split(data):
    
    n = data.shape[0]
    
    train_df = data.iloc[0: n * 70 //100] # "iloc" because we have to select the lines at the indicies 0 to int(n*0.7) compared to "loc"
    val_df = data.iloc[n * 70 //100 : n * 90 //100]
    test_df = data.iloc[n * 90 //100:]
    
    return(train_df, val_df, test_df)

def naive_eval_arr(X, y, lookback, delay):
    batch_maes = []
    for i in range(0, len(X)):
        preds = X[i, -1, 1] #For all elements in the batch, we are saying the prediction of temperature is equal to the last temperature recorded within the 48 hours
        mae = np.mean(np.abs(preds - y[i]))
        batch_maes.append(mae)
    return(np.mean(batch_maes))

def split_sequences(sequences, n_steps, extra_lag=False, long_lag_step=7, max_step=30, idx=0, multivar=False):
    #if not adding extra lag features adjust max_step and n_steps to aling
    if not extra_lag:
        max_step=n_steps
        n_steps+=1
        
    
    X, y = list(), list()
    for i in range(len(sequences)):
        
        # find the end of this pattern
        #end_ix = i + n_steps
        end_ix = i + max_step
        
        #create a list with the indexes we want to include in each sample
        slices = [x for x in range(end_ix-1,end_ix-n_steps, -1)] + [y for y in range(end_ix-n_steps, i, -long_lag_step)]
        
        #reverse the slice indexes
        slices = list(reversed(slices))
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break


        # gather input and output parts of the pattern
        seq_x = sequences[slices, :]
        seq_y = sequences[end_ix, :]

        X.append(seq_x)
        y.append(seq_y)
        
    X = np.array(X)
    y = np.array(y)
    
    if multivar:
        #unstack the 3rd dimension and select the first element(energy load)
        y = y[:,idx]
    
    return X, y

def generator(data, lookback, delay, min_index, max_index, shuffle=True, batch_size=32, step=1):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while True:
        if shuffle == True: # pay attention ! We are not shuffeling timesteps but elemnts within a batch ! It is important to keep the data in time order
            rows = np.random.randint(min_index + lookback, max_index-delay-1, size=batch_size) # return an array containing size elements ranging from min_index+lookback to max_index
        else:
            if i + batch_size >= max_index-delay-1: #Since we are incrementing on "i". If its value is greater than the max_index --> start from the begining
                i = min_index + lookback # We need to start from the indice lookback, since we want to take lookback elements here.
            rows = np.arange(i, min(i + batch_size, max_index)) # Just creating an array that contain the indices of each sample in the batch
            i+=len(rows) # rows represents the number of sample in one batch
            
        samples = np.zeros((len(rows), lookback//step, data.shape[-1])) # shape = (batch_size, lookback, nbr_of_features)
        targets = np.zeros((len(rows),)) #Shape = (batch_size,)
        
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step) #From one indice given by rows[j], we are picking loockback previous elements in the dataset
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1] #We only want to predict the temperature for now,since [1], the second column
        return samples, targets # The yield that replace the return to create a generator and not a regular function.
        
def evaluate_timeseries_model(df, n_folds=5, multivar=False):
    scores, histories = list(), list()
    df_processed = preprocessing(df)
    train_df, val_df, test_df = split(df_processed)
    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean)/train_std # As simple as that !
    val_df = (val_df - train_mean)/train_std
    test_df = (test_df - train_mean)/train_std
    lookback = 48 # Looking at all features for the past 2 days
    delay = 24 # Trying to predict the temperature for the next day
    batch_size = 64 # Features will be batched 32 by 32.
    X_train, Y_train = create_dataset(train_df, train_df['T (degC)'], delay = delay, lookback = lookback)
    X_val, Y_val = create_dataset(val_df, val_df['T (degC)'], delay = delay)
    naive_loss_arr = naive_eval_arr(X_val, Y_val, lookback = lookback, delay = delay)

    naive_loss_arr = round(naive_eval_arr(X_val, Y_val, lookback = lookback, delay = delay),2) # Round the value
    

    data_train = train_df.to_numpy()
    (X_train, Y_train) = generator(data = data_train, lookback = lookback, delay =delay, min_index = 0, 
                                   max_index = len(data_train), shuffle = True, batch_size = batch_size)

    data_val = val_df.to_numpy()
    (X_val, Y_val) = generator(data = data_val, lookback = lookback, delay =delay, min_index = 0, 
                               max_index = len(data_val), batch_size = batch_size)

    data_test = test_df.to_numpy()
    test_gen = generator(data = data_val, lookback = lookback, delay =delay, min_index = 0,
                         max_index = len(data_test), batch_size = batch_size)
    tscv = TimeSeriesSplit(n_splits=5, test_size=2)
    
    '''for train_index, test_index in tscv.split(X_train):
        X_train, X_test = X_train[train_index], X_train[test_index]
        Y_train, Y_test = Y_train[train_index], Y_train[test_index]'''
    print(naive_loss_arr)
    lr = 3e-4
    n_steps=48#24*30
    n_horizon=24
    #batch_size=64
    if multivar:
        n_features=5
    else:
        n_features=1
    model=vtoc_cnn_model(n_steps, n_horizon, n_features, lr)

    print(X_train.shape, Y_train.shape)
    model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, verbose=3, shuffle=True)
    #model.fit(X_train, X_train, epochs=10, verbose=3, shuffle=True)
    train_encoded = model.predict(X_train)
    validation_encoded = model.predict(X_val)
    print('Encoded time-series shape', train_encoded.shape)
    print('Encoded time-series sample', train_encoded[0])
    return model.evaluate(X_test, Y_test)


In [28]:
import pandas as pd
import numpy as np
#csv_path = "/kaggle/input/energy-consumption-generation-prices-and-weather/energy_dataset.csv"
timeseries_df = pd.read_csv(csv_path)
#evaluate_timeseries_model(timeseries_df, n_folds=5, multivar=False)

# Regression

In [187]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.model_selection import train_test_split
def vtoc_regression_model(norm, model_type):
    if model_type=='linear':
        model = Sequential()
        #model.add(Flatten())
        model.add(Dense(1))
    else:
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1))

    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')

    return model


def eval_regression_data(dataset, target):
    dataset = dataset.astype('float32')
    dataset.isna().sum()
    dataset=dataset.dropna()
    #dataset['Origin'] = dataset['Origin'].map({1.0: 'USA', 2.0: 'Europe', 3.0: 'Japan'})
    Y=dataset[target]
    X=dataset.loc[:, dataset.columns != target]
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3)
    horsepower = np.array(X_train['Horsepower']).astype('float32')

    horsepower_normalizer = Normalization(input_shape=[1,], axis=None)
    horsepower_normalizer.adapt(horsepower)
    
    linear_model=vtoc_regression_model(horsepower_normalizer, model_type='linear')
    history = linear_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, verbose=2)
    test_results = linear_model.predict(X_test)
    return test_results

In [188]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)
eval_regression_data(raw_dataset, 'MPG')


Epoch 1/100
9/9 - 0s - loss: 655.3945 - val_loss: 324.5295
Epoch 2/100
9/9 - 0s - loss: 177.0524 - val_loss: 34.7252
Epoch 3/100
9/9 - 0s - loss: 61.0334 - val_loss: 41.9099
Epoch 4/100
9/9 - 0s - loss: 27.7987 - val_loss: 17.4146
Epoch 5/100
9/9 - 0s - loss: 19.5658 - val_loss: 30.1306
Epoch 6/100
9/9 - 0s - loss: 31.2055 - val_loss: 38.6909
Epoch 7/100
9/9 - 0s - loss: 35.5456 - val_loss: 29.2807
Epoch 8/100
9/9 - 0s - loss: 35.7468 - val_loss: 31.9856
Epoch 9/100
9/9 - 0s - loss: 32.8222 - val_loss: 39.2455
Epoch 10/100
9/9 - 0s - loss: 33.6606 - val_loss: 10.7386
Epoch 11/100
9/9 - 0s - loss: 34.6217 - val_loss: 25.9623
Epoch 12/100
9/9 - 0s - loss: 35.2077 - val_loss: 23.6323
Epoch 13/100
9/9 - 0s - loss: 33.2501 - val_loss: 29.8130
Epoch 14/100
9/9 - 0s - loss: 31.0524 - val_loss: 11.1152
Epoch 15/100
9/9 - 0s - loss: 33.3188 - val_loss: 18.9865
Epoch 16/100
9/9 - 0s - loss: 30.4590 - val_loss: 8.6874
Epoch 17/100
9/9 - 0s - loss: 8.8491 - val_loss: 9.0524
Epoch 18/100
9/9 - 0s -

array([[77.469696],
       [71.14866 ],
       [76.31318 ],
       [68.196205],
       [72.368904],
       [84.514786],
       [70.25062 ],
       [73.09792 ],
       [72.51673 ],
       [67.94489 ],
       [68.79024 ],
       [65.599   ],
       [67.697586],
       [71.97364 ],
       [80.36729 ],
       [73.08812 ],
       [68.25088 ],
       [78.07401 ],
       [68.34635 ],
       [79.209366],
       [66.51295 ],
       [79.367584],
       [70.48836 ],
       [71.486885],
       [75.88871 ],
       [68.15867 ],
       [80.30823 ],
       [76.226944],
       [77.32481 ],
       [70.72942 ],
       [71.44613 ],
       [69.95439 ],
       [74.997086],
       [66.39675 ],
       [71.32725 ],
       [72.909065],
       [75.04423 ],
       [70.54749 ],
       [71.16452 ],
       [72.27059 ],
       [73.52431 ],
       [68.89938 ],
       [70.25947 ],
       [76.23491 ],
       [70.71241 ],
       [73.79117 ],
       [84.260574],
       [81.681915],
       [72.273   ],
       [74.44572 ],


In [ ]:
char_arr = [c for c in "abcdefghijklmnopqrstuvwxyz0"]
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
n_class = len(word_dict)
seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

n_step = 3
n_hidden = 128

inputs = [sen[:3] for sen in seq_data]
input_batch, target_batch = make_batch(seq_data)
predict =  sess.run([prediction], feed_dict={X: input_batch})
print(inputs, '->', [number_dict[n] for n in predict[0]])

def make_batch(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]]
        target = word_dict[seq[-1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])
        #print(seq, len(seq))

    return input_batch, target_batchtf.reset_default_graph()

# Model
X = tf.placeholder(tf.float32, [len(seq_data), n_step, n_class]) # [batch_size, n_step, n_class]
Y = tf.placeholder(tf.float32, [len(seq_data), n_class])         # [batch_size, n_class]
print(X.shape, Y.shape)
W = tf.Variable(tf.random_normal([n_hidden, n_step]))
b = tf.Variable(tf.random_normal([len(seq_data), n_class]))
print(W.shape)
cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# outputs : [batch_size, n_step, n_hidden]
outputs = tf.transpose(outputs, [0, 1, 2]) # [n_step, batch_size, n_hidden]
outputs = outputs[-1] # [batch_size, n_hidden]
print(outputs.shape)
model = tf.reshape(tf.matmul(outputs, new_W),[9]) + new_b # model : [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(.5).minimize(cost)

prediction = tf.cast(tf.argmax(model, 1), tf.int32)


# Build Network from scratch

In [113]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def create_dataset(xs, ys, n_classes=10):
    xs = tf.cast(xs, tf.float32) / 255.0
    ys = tf.cast(ys, tf.float32)
    ys = tf.one_hot(ys, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((xs, ys)).map(preprocess).shuffle(len(ys)).batch(128)

def val_nn(training_inputs_data, training_outputs_data, test_inputs):
    tf.compat.v1.disable_eager_execution()
    training_inputs = tensorflow.compat.v1.placeholder(shape=[None, 3], dtype=tensorflow.float32)  
    training_outputs = tensorflow.compat.v1.placeholder(shape=[None, 1], dtype=tensorflow.float32) #Desired outputs for each input  
    weights = tensorflow.Variable(initial_value=[[.3], [.1], [.8]], dtype=tensorflow.float32)  
    bias = tensorflow.Variable(initial_value=[[1]], dtype=tensorflow.float32)  

    af_input = tensorflow.matmul(training_inputs, weights) + bias  
  
    # Activation function of the output layer neuron  
    predictions = tensorflow.nn.sigmoid(af_input)  
    # Measuring the prediction error of the network after being trained  
    prediction_error = tensorflow.reduce_sum(training_outputs - predictions)  
    # Minimizing the prediction error using gradient descent optimizer  
    
    train_op = tensorflow.compat.v1.train.GradientDescentOptimizer(learning_rate=0.05).minimize(prediction_error) 
    # Creating a TensorFlow Session  
    sess = tensorflow.compat.v1.Session()  
    # Initializing the TensorFlow Variables (weights and bias)  
    sess.run(tensorflow.compat.v1.global_variables_initializer())  
    
    # Training loop of the neural network  
    for step in range(10000):  
        sess.run(fetches=train_op, feed_dict={training_inputs: training_inputs_data, training_outputs: training_outputs_data})  
        # Class scores of some testing data  
    score= sess.run(fetches=predictions, feed_dict={training_inputs: [[248, 80, 68], [0, 0, 255]]})
    predictions =  sess.run(fetches=test_inputs)
    # Closing the TensorFlow Session to free resources  
    sess.close()
    return predictions

In [114]:
import tensorflow
X_train = [[255, 0, 0], [248, 80, 68], [0, 0, 255],[67, 15, 210]]  
Y_train = [[1], [1], [0], [0]] 
X_test = [[5, 2, 13], [7, 9, 0]]
X_test=tensorflow.convert_to_tensor(value=X_test, dtype=tensorflow.float32)
val_nn(X_train, Y_train, X_test)

array([[ 5.,  2., 13.],
       [ 7.,  9.,  0.]], dtype=float32)